In [1]:
import matplotlib.pyplot as plt
import seaborn as sns

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler
import descartes
import geopandas as gpd
from shapely.geometry import Point, Polygon
import folium
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

In [42]:
df=pd.read_csv('merged.csv')

In [43]:
pd.set_option('display.max_columns', None)


In [44]:
df=pd.DataFrame(df)
df.head(20)
drop= df['inspector'].value_counts() < 20

In [45]:
df= df.drop(df[['transactions', 'price', 'establishment_x', 'establishment_y', 'address_x', 'address_y', 'trimmed_name_x', 'trimmed_name_y', 'clean_address_x', 'clean_address_y', 'address_distance', 'name_distance']], axis=1)

In [46]:
#Dropped all establishments with less than 22 reviews
df=  df[df['review_count'] >= 22] 
df.describe()

,Unnamed: 0,number_of_violations,zip_code,house_numbers,distance,review_count,longitude,latitude,sector,total_score,rating
count,1820.000000,1820.000000,1820.000000,1820.00000,1820.000000,1820.000000,1820.000000,1820.000000,1820.000000,1820.000000,1820.000000
mean,1166.184066,5.082967,78220.814286,4784.15989,7830.619272,210.672527,-98.514171,29.489581,15.985714,93.996154,3.678571
std,709.624182,4.892942,15.141757,4756.65303,3071.065979,288.162365,0.058245,0.055954,9.218967,5.436068,0.669349
min,0.000000,0.000000,78201.000000,21.00000,811.199667,22.000000,-98.708442,29.354318,1.000000,56.000000,1.000000
25%,529.500000,2.000000,78209.000000,821.00000,5929.771588,55.000000,-98.564447,29.433901,8.000000,91.000000,3.500000
50%,1171.500000,4.000000,78216.000000,2918.00000,7986.163583,109.000000,-98.495517,29.492450,16.000000,95.000000,4.000000
75%,1782.250000,7.000000,78230.000000,8226.50000,9377.441334,251.000000,-98.480431,29.530199,22.000000,98.000000,4.000000
max,2408.000000,50.000000,78259.000000,22211.00000,20222.717722,2661.000000,-98.398119,29.661520,35.000000,100.000000,5.000000


In [47]:
#Dropped all establishments with less than 22 reviews
df=  df[df['review_count'] >= 22] 

In [48]:
#grouped health scores into 5 groups with a range of three
health_scores= {'health_scores' : []}
for x in df['total_score']:
#     print(x)
    if x > 95:
        health_scores['health_scores'].append('1')
    elif x> 90:
        health_scores['health_scores'].append('2')
    else:
        health_scores['health_scores'].append('3')       
#df
df['health_scores']= health_scores['health_scores']  


In [49]:
#df.isnull().sum()
#len(health_scores)
#len(df)
df['inspector'].value_counts()

Lauren Schumacher     180
Jeffrey Carrizales    136
RYAN DAVIS            129
Jacob Johanson        128
Sumner Bumgardner     117
Bamba Njie             89
Diana Garcia           88
Deborah Liu            82
Bernard Capdeboscq     79
Cicily Martinez        77
Valerie Suarez         75
Benito Zuniga          72
Daisy Zamora           68
Nathan Balfour         64
Raul Jimenez           38
John Payelle           38
William Hudson         37
Kassandra Aguilar      35
Debra Hernandez        31
Mario Cavazos          27
Erica Arreola          27
MICHELLE MORENO        26
Kathleen Prenzler      26
William Emminger       26
Amanda Cantu           22
Kacie Patrick          20
Adan Zamora            18
Sean Gilbert           16
Fermin Garza           15
Monica Caballero        7
Victoria Paredes        7
Juanicia Page           6
Joe Mokry               5
Daniel Guzman           3
Maurico Ripley          3
Erica Llanas            2
Rebecca Vera            1
Name: inspector, dtype: int64

In [50]:
vdf = pd.DataFrame(df['inspector'].value_counts())
vdf['check'] = vdf['inspector'] > 1
over_20 = list(vdf.loc[vdf['check'], :].reset_index()['index'])

In [51]:
inspectors = pd.DataFrame(df.inspector.unique())

In [52]:
df['check'] = df['inspector'].isin(over_20)
new_df = df.loc[df['check'], :]
new_df = new_df.drop(columns = ['check'])
new_df

,Unnamed: 0,inspection_date,inspector,number_of_violations,violations,zip_code,house_numbers,categories,distance,review_count,longitude,latitude,name,sector,district,total_score,rating,health_scores
0,0,2019-01-02,Cicily Martinez,3,"['22', '29', '45']",78212,2524,"['Italian', 'Desserts', 'Seafood']",4950.353662,99.0,-98.493780,29.453730,Capparelli's On Main,35,D,97,3.5,1
1,1,2019-03-06,RYAN DAVIS,1,['31'],78230,14439,"['French', 'Asian Fusion', 'Tapas/Small Plates']",9312.955170,102.0,-98.544540,29.573330,Frederick's Bistro,9,A,98,4.0,1
3,3,2019-11-04,Cicily Martinez,3,"['10', '15', '40']",78201,914,['Mexican'],3194.340357,42.0,-98.508619,29.466156,Fina's Kitchen,35,D,93,4.0,2
4,4,2019-03-14,RYAN DAVIS,0,[],78230,13151,['Barbeque'],7935.733616,41.0,-98.537950,29.562850,Bill Miller Bar-B-Q,9,A,100,2.5,1
5,5,2019-03-14,RYAN DAVIS,1,['21'],78230,10003,['American (Traditional)'],6996.407916,434.0,-98.536348,29.554391,Green Vegetarian Cuisine,9,A,98,4.5,1
6,6,2019-03-14,RYAN DAVIS,1,['18'],78230,10003,['Vietnamese'],6937.261517,70.0,-98.534051,29.554894,Pho VN Bistro,9,A,97,3.5,1
7,7,2019-07-05,Cicily Martinez,3,"['1', '39', '45']",78201,919,"['Tex-Mex', 'Breakfast & Brunch']",3138.557809,145.0,-98.508632,29.466663,Rolando's Super Tacos No 1,35,D,95,3.5,2
8,8,2019-07-05,Cicily Martinez,3,"['1', '39', '45']",78201,919,"['Tex-Mex', 'Breakfast & Brunch']",3138.557809,145.0,-98.508632,29.466663,Rolando's Super Tacos No 1,35,D,95,3.5,2
9,9,2019-07-05,RYAN DAVIS,0,[],78230,10003,['American (Traditional)'],6996.407916,434.0,-98.536348,29.554391,Green Vegetarian Cuisine,9,A,100,4.5,1
10,10,2019-07-05,Cicily Martinez,3,"['9', '37', '40']",78201,914,['Mexican'],3194.340357,42.0,-98.508619,29.466156,Fina's Kitchen,35,D,95,4.0,2


In [53]:
new_df.describe()

,Unnamed: 0,number_of_violations,zip_code,house_numbers,distance,review_count,longitude,latitude,sector,total_score,rating
count,1819.000000,1819.000000,1819.000000,1819.000000,1819.000000,1819.000000,1819.000000,1819.000000,1819.000000,1819.000000,1819.00000
mean,1166.623969,5.083562,78220.822980,4786.601979,7830.658386,210.773502,-98.514180,29.489616,15.979659,93.996701,3.67867
std,709.571052,4.894222,15.141376,4756.819669,3071.910037,288.209398,0.058260,0.055949,9.217881,5.437513,0.66952
min,0.000000,0.000000,78201.000000,21.000000,811.199667,22.000000,-98.708442,29.354318,1.000000,56.000000,1.00000
25%,530.500000,2.000000,78209.000000,822.500000,5929.771588,55.000000,-98.564562,29.434249,8.000000,91.000000,3.50000
50%,1172.000000,4.000000,78216.000000,2918.000000,7986.163583,109.000000,-98.495517,29.492450,16.000000,95.000000,4.00000
75%,1782.500000,7.000000,78230.000000,8229.000000,9377.441334,251.000000,-98.480431,29.530199,22.000000,98.000000,4.00000
max,2408.000000,50.000000,78259.000000,22211.000000,20222.717722,2661.000000,-98.398119,29.661520,35.000000,100.000000,5.00000


In [54]:
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import export_graphviz
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

In [15]:
#training / test split
X= new_df[['number_of_violations', 'review_count', 'rating']]
y= new_df['health_scores']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = .30, random_state = 123, stratify=new_df.health_scores)

X_train.head()

,number_of_violations,review_count,rating
2021,0,193.0,4.0
186,2,418.0,3.5
1068,4,238.0,4.0
513,4,22.0,3.0
1436,5,118.0,3.0


In [16]:
# for classification you can change the algorithm to gini or entropy (information gain).  Default is gini.
clf = DecisionTreeClassifier(criterion='entropy', max_depth=5, random_state=123)

In [17]:
#Fit the model to the training data

clf.fit(X_train, y_train)

DecisionTreeClassifier(class_weight=None, criterion='entropy', max_depth=5,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=123, splitter='best')

In [18]:
#Estimate health group

y_pred = clf.predict(X_train)
y_pred[1:5]

array(['1', '1', '2', '2'], dtype=object)

In [19]:
#Estimate the probability of a health group
y_pred_proba = clf.predict_proba(X_train)
y_pred_proba

array([[0.91954023, 0.08045977, 0.        ],
       [0.82352941, 0.16806723, 0.00840336],
       [0.47321429, 0.46428571, 0.0625    ],
       ...,
       [0.16129032, 0.55913978, 0.27956989],
       [0.03703704, 0.03703704, 0.92592593],
       [0.82352941, 0.16806723, 0.00840336]])

In [20]:
confusion_matrix(y_train, y_pred)

array([[533,  51,  14],
       [140, 206,  44],
       [ 22,  80, 183]])

In [21]:
#Accuracy: number of correct predictions over the number of total instances that have been evaluated.
print('Accuracy of Decision Tree classifier on training set: {:.2f}'
     .format(clf.score(X_train, y_train)))

Accuracy of Decision Tree classifier on training set: 0.72


In [22]:
#Test Model

#Compute the accuracy of the model when run on the test data

print('Accuracy of Decision Tree classifier on test set: {:.2f}'
     .format(clf.score(X_test, y_test)))

Accuracy of Decision Tree classifier on test set: 0.69


In [23]:
print(classification_report(y_train, y_pred))

              precision    recall  f1-score   support

           1       0.77      0.89      0.82       598
           2       0.61      0.53      0.57       390
           3       0.76      0.64      0.70       285

    accuracy                           0.72      1273
   macro avg       0.71      0.69      0.70      1273
weighted avg       0.72      0.72      0.72      1273



In [24]:
clf

DecisionTreeClassifier(class_weight=None, criterion='entropy', max_depth=5,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=123, splitter='best')

In [25]:
#Encoding

In [55]:
#Label Encoding the inspector column
label_enc= LabelEncoder()
new_df['inspector_encode'] = label_enc.fit_transform(new_df.inspector)


In [29]:
#new_df['inspector_encode'].max()

35

In [33]:
#print(len(X[0]))

36


In [56]:
#OneHot Encoding
one_enc = OneHotEncoder()
X = one_enc.fit_transform(new_df.inspector_encode.values.reshape(-1,1)).toarray()
print(X)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


/usr/local/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


In [89]:
#Joining the encoded data to the main Df
dfOneHot = pd.DataFrame(X, columns = ["health_inspector_"+str(int(i)) for i in range(X.shape[1])])
#print(len(dfOneHot))
#print(len(new_df))
#new_df = pd.concat([new_df, dfOneHot], axis=1)
#new_df=new_df.drop(columns=['inspector_0', 'inspector_1'])
#len(pd.concat([new_df, dfOneHot], axis = 1))
#dfOneHot = dfOneHot.reset_index()
#print(dfOneHot)
#new_df = new_df.reset_index()
#new_df = new_df.drop(columns = ['Unnamed: 0'])
#new_df
#new_one = dfOneHot.reset_index()
#new_one
#new_df

#final = new_one.join(new_df, on = 'index')

1819
1819


In [108]:
#new_df = new_df.reset_index()
#new_df['index']= new_df['level_0']
#new_df['index']
#dfOneHot = dfOneHot.reset_index()
#dfOneHot['index']
new_df.join(dfOneHot, on = 'index', lsuffix='_left', rsuffix='_right')

,level_0,index_left,inspection_date,inspector,number_of_violations,violations,zip_code,house_numbers,categories,distance,review_count,longitude,latitude,name,sector,district,total_score,rating,health_scores,inspector_encode,index_right,health_inspector_0,health_inspector_1,health_inspector_2,health_inspector_3,health_inspector_4,health_inspector_5,health_inspector_6,health_inspector_7,health_inspector_8,health_inspector_9,health_inspector_10,health_inspector_11,health_inspector_12,health_inspector_13,health_inspector_14,health_inspector_15,health_inspector_16,health_inspector_17,health_inspector_18,health_inspector_19,health_inspector_20,health_inspector_21,health_inspector_22,health_inspector_23,health_inspector_24,health_inspector_25,health_inspector_26,health_inspector_27,health_inspector_28,health_inspector_29,health_inspector_30,health_inspector_31,health_inspector_32,health_inspector_33,health_inspector_34,health_inspector_35
0,0,0,2019-01-02,Cicily Martinez,3,"['22', '29', '45']",78212,2524,"['Italian', 'Desserts', 'Seafood']",4950.353662,99.0,-98.493780,29.453730,Capparelli's On Main,35,D,97,3.5,1,5,0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,1,2019-03-06,RYAN DAVIS,1,['31'],78230,14439,"['French', 'Asian Fusion', 'Tapas/Small Plates']",9312.955170,102.0,-98.544540,29.573330,Frederick's Bistro,9,A,98,4.0,1,28,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2,2,2019-11-04,Cicily Martinez,3,"['10', '15', '40']",78201,914,['Mexican'],3194.340357,42.0,-98.508619,29.466156,Fina's Kitchen,35,D,93,4.0,2,5,2,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,3,3,2019-03-14,RYAN DAVIS,0,[],78230,13151,['Barbeque'],7935.733616,41.0,-98.537950,29.562850,Bill Miller Bar-B-Q,9,A,100,2.5,1,28,3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4,4,2019-03-14,RYAN DAVIS,1,['21'],78230,10003,['American (Traditional)'],6996.407916,434.0,-98.536348,29.554391,Green Vegetarian Cuisine,9,A,98,4.5,1,28,4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,5,5,2019-03-14,RYAN DAVIS,1,['18'],78230,10003,['Vietnamese'],6937.261517,70.0,-98.534051,29.554894,Pho VN Bistro,9,A,97,3.5,1,28,5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,6,6,2019-07-05,Cicily Martinez,3,"['1', '39', '45']",78201,919,"['Tex-Mex', 'Breakfast & Brunch']",3138.557809,145.0,-98.508632,29.466663,Rolando's Super Tacos No 1,35,D,95,3.5,2,5,6,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,7,7,2019-07-05,Cicily Martinez,3,"['1', '39', '45']",78201,919,"['Tex-Mex', 'Breakfast & Brunch']",3138.557809,145.0,-98.508632,29.466663,Rolando's Super Tacos No 1,35,D,95,3.5,2,5,7,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,8,8,2019-07-05,RYAN DAVIS,0,[],78230,10003,['American (Traditional)'],6996.407916,434.0,-98.536348,29.554391,Green Vegetarian Cuisine,9,A,100,4.5,1,28,8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,9,9,2019-07-05,Cicily Martinez,3,"['9', '37', '40']",78201,914,['Mexican'],3194.340357,42.0,-98.508619,29.466156,Fina's Kitchen,35,D,95,4.0,2,5,9,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [109]:
new_df.dropna()
new_df.isnull().sum()

level_0                 0
index                   0
inspection_date         0
inspector               0
number_of_violations    0
violations              0
zip_code                0
house_numbers           0
categories              0
distance                0
review_count            0
longitude               0
latitude                0
name                    0
sector                  0
district                0
total_score             0
rating                  0
health_scores           0
inspector_encode        0
dtype: int64

In [112]:
#### Decision tree including number of review, total violations, yelp score, and inspector
#training / test split
X1= new_df.drop(columns=['inspection_date', 'inspector', 'violations', 'zip_code', 'house_numbers', 'categories', 'distance', 'longitude', 'latitude', 'name', 'sector', 'district', 'health_scores'])
y1= new_df['health_scores']

X_train1, X_test1, y_train1, y_test1 = train_test_split(X1, y1, test_size = .30, random_state = 123, stratify=new_df.health_scores)
X_train1.head()

,level_0,index,number_of_violations,review_count,total_score,rating,inspector_encode
1532,1532,1532,0,193.0,100,4.0,14
163,163,163,2,418.0,96,3.5,27
846,846,846,4,238.0,93,4.0,14
442,442,442,4,22.0,94,3.0,31
1109,1109,1109,5,118.0,90,3.0,8


In [120]:
# for classification you can change the algorithm to gini or entropy (information gain).  Default is gini.
clf1 = DecisionTreeClassifier(criterion='entropy', max_depth=5, random_state=123)

In [121]:
clf1.fit(X_train1, y_train1)

NameError: name 'X_train1' is not defined

In [122]:
#Estimate health group

y_pred1 = clf1.predict(X_train1)
y_pred1[1:5]

NameError: name 'X_train1' is not defined

In [123]:
#Estimate the probability of a health group
y_pred_proba1 = clf1.predict_proba1(X_train1)
y_pred_proba1

AttributeError: 'DecisionTreeClassifier' object has no attribute 'predict_proba1'

In [124]:
confusion_matrix(y_train1, y_pred1)

NameError: name 'y_train1' is not defined

In [119]:
#Accuracy: number of correct predictions over the number of total instances that have been evaluated.
print('Accuracy of Decision Tree classifier on training set: {:.2f}'
     .format(clf1.score(X_train1, y_train1)))

Accuracy of Decision Tree classifier on training set: 1.00
